<a href="https://colab.research.google.com/github/Tajyu/Master_Degree_Paper/blob/main/Code/GUI/TKinter_GUI_Image_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# VScode에서 사용 가능.
import os
import sys
import tkinter.ttk as ttk
from tkinter import *
from tkinter import filedialog # 서브 모델이라서 추가적으로 import 해줘야함.
import tkinter.messagebox as msgbox
from PIL import Image
import matplotlib.pyplot as plt 
import cv2
import numpy as np

root = Tk()
root.title("Algae Dictionary")
root.geometry("700x650")

# 파일 추가
def add_file():
    files = filedialog.askopenfilenames(title = "이미지 파일 선택", \
        filetypes = (("PNG 파일", "*.png"), ("모든 파일", "*.*")), \
        initialdir = "C:/") # 최초의 경로를 C:/ 경로로 보여준다.

    # 사용자가 선택한 파일 목록 보여준다.
    for file in files:
        list_file.insert(END, file)

def del_file():
    for index in reversed(list_file.curselection()):
        list_file.delete(index)

# 파일 저장 경로(폴더)
def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected == "": # 파일 저장 경로를 정하지 않고 취소를 누를때
        print("폴더 선택 취소")
        return
    txt_dest_path.delete(0, END)
    txt_dest_path.insert(0, folder_selected)


# 이미지 전처리(Gaussian Binary)
def img_processing():
    images = []
    for i in range(list_file.size()):
        images.append(cv2.imread(list_file.get(i), flags=cv2.IMREAD_GRAYSCALE))
    idx = 0
    
    for img in images:
        height, width = img.shape
        block_size = 5
        C = 5

        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        img_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, C)

        imgTopHat = cv2.morphologyEx(img_thresh, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(img_thresh, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(img_thresh, imgTopHat)
        img = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)
        
        img_blurred = cv2.GaussianBlur(img_thresh, ksize=(5, 5), sigmaX=0)

        contours, _ = cv2.findContours(img_thresh, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE)
        
        # temp_result = np.zeros((height, width), dtype=np.uint8)

        cv2.drawContours(img_thresh, contours = contours, contourIdx = -1, color=(255, 255, 255))

        cv2.imwrite(txt_dest_path.get() + '/contours_image_' + str(idx) + '.PNG', img_thresh)
        
        # 포맷
        img_format = cmb_format.get().lower() # PNG를 소문자로 변경

        progress = (idx + 1) / len(images) * 100 # 실제 percent 정보를 계산
        p_var.set(progress)
        progress_bar.update()
        idx = idx + 1
   


# 시작 버튼 누를시에 실행
def start():
    # 각 옵션 확인
    # print("이미지 전처리: ", cmb_processing.get())
    # print("파일 형식 지정: ", cmb_format.get())

    # 파일 목록 확인
    if list_file.size() == 0:
        msgbox.showwarning("이미지 없음", "이미지 파일을 추가하세요.")
        return

    # 저장 경로 확인
    if len(txt_dest_path.get()) == 0:
        msgbox.showwarning("저장 경로 없음", "저장 경로를 설정하세요.")
        return

    # 이미지 전처리(Gaussian Binary) 실행
    img_processing()


# 파일 프레임 (파일 추가, 선택 삭제)
file_frame = Frame(root)
file_frame.pack(fill="x", padx=5, pady=5) # 간격 띄우기
 
btn_add_file = Button(file_frame, padx=5, pady=5, width=12, text="파일 추가", command=add_file)
btn_add_file.pack(side="left")

btn_del_file = Button(file_frame, padx=5, pady=5, width=12, text="선택 삭제", command=del_file)
btn_del_file.pack(side="right")

# 리스트 프레임
list_frame = Frame(root)
list_frame.pack(fill="both", padx=5, pady=5)

scrollbar = Scrollbar(list_frame)
scrollbar.pack(side="right", fill="y")

list_file = Listbox(list_frame, selectmode="extended", height=20, yscrollcommand=scrollbar.set)
list_file.pack(side="left", fill="both", expand=True)
scrollbar.config(command=list_file.yview)

# 저장 경로 프레임
path_frame = LabelFrame(root, text="저장 경로")
path_frame.pack(fill="x", padx=5, pady=5, ipady=5)

txt_dest_path = Entry(path_frame)
txt_dest_path.pack(side="left", fill="x", expand=True, padx=5, pady=5, ipady=4) # 높이 변경

btn_dest_path = Button(path_frame, text="찾아보기", width=10, command=browse_dest_path)
btn_dest_path.pack(side="right", padx=5, pady=5)

# 옵션 프레임
frame_option = LabelFrame(root, text="옵션")
frame_option.pack(padx=5, pady=5, ipady=5)

# 1. 이미지 이진화
# 이미지 이진화 레이블
lbl_binary = Label(frame_option, text="이미지 이진화", width=10)
lbl_binary.pack(side="left", padx=5, pady=5)

# 이미지 이진화 선택
opt_processing = ["이미지 전처리"]
cmb_processing = ttk.Combobox(frame_option, state="readonly", values=opt_processing, width=12)
cmb_processing.current(0)
cmb_processing.pack(side="left", padx=10, pady=5)

# 2. 파일 저장 형식  포맷 옵션
# 파일 저장 형식 옵션 레이블
lbl_format = Label(frame_option, text="저장 형식", width=6)
lbl_format.pack(side="left", padx=5, pady=5)

# 파일 포맷 옵션 콤보
opt_format = ["PNG"]
cmb_format = ttk.Combobox(frame_option, state="readonly", values=opt_format, width=10)
cmb_format.current(0)
cmb_format.pack(side="left", padx=5, pady=5)

# 진행 상황 Progress Bar
frame_progress = LabelFrame(root, text="진행 상황")
frame_progress.pack(fill="x", padx=5, pady=5, ipady=5)

p_var = DoubleVar()
progress_bar = ttk.Progressbar(frame_progress, maximum=100, variable=p_var)
progress_bar.pack(fill="x", padx=5, pady=5)

# 실행 프레임
frame_run = Frame(root)
frame_run.pack(fill="x", padx=5, pady=5)

btn_close = Button(frame_run, padx=5, pady=5, text="닫기", width=12, command=root.quit)
btn_close.pack(side="right", padx=5, pady=5)

# 시작이랑 종료 둘 다 right로해서 반대로 출력되기때문에 시작을 뒤에 작성
btn_start = Button(frame_run, padx=5, pady=5, text="시작", width=12, command=start)
btn_start.pack(side="right", padx=5, pady=5)

root.resizable(False, False)
root.mainloop()